<a href="https://colab.research.google.com/github/jamesjykim2/neural-network-challenge-2/blob/main/attrition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Part 1: Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras.layers import Embedding, Dense, Input, Dropout, LSTM, Activation, Conv2D, Reshape, Average, Bidirectional

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [2]:
# Determine the number of unique values in each column
attrition_df.nunique()


,0
Age,43
Attrition,2
BusinessTravel,3
Department,3
DistanceFromHome,29
Education,5
EducationField,6
EnvironmentSatisfaction,4
HourlyRate,71
JobInvolvement,4


In [3]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]

In [4]:
# Create a list of at least 10 column names to use as X data
X_data = ['Age', 'DistanceFromHome', 'OverTime', 'Education','HourlyRate', 'PerformanceRating', 'JobSatisfaction', 'YearsAtCompany', 'NumCompaniesWorked', 'TotalWorkingYears']

# Create X_df using your selected columns
X_df = attrition_df[X_data]

# Show the data types for X_df
X_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Age                 1470 non-null   int64 
 1   DistanceFromHome    1470 non-null   int64 
 2   OverTime            1470 non-null   object
 3   Education           1470 non-null   int64 
 4   HourlyRate          1470 non-null   int64 
 5   PerformanceRating   1470 non-null   int64 
 6   JobSatisfaction     1470 non-null   int64 
 7   YearsAtCompany      1470 non-null   int64 
 8   NumCompaniesWorked  1470 non-null   int64 
 9   TotalWorkingYears   1470 non-null   int64 
dtypes: int64(9), object(1)
memory usage: 115.0+ KB


In [5]:
X_df.head()

,Age,DistanceFromHome,OverTime,Education,HourlyRate,PerformanceRating,JobSatisfaction,YearsAtCompany,NumCompaniesWorked,TotalWorkingYears
0,41,1,Yes,2,94,3,4,6,8,8
1,49,8,No,1,61,4,2,10,1,10
2,37,2,Yes,2,92,3,3,0,6,7
3,33,3,Yes,4,56,3,3,8,1,8
4,27,2,No,1,40,3,2,2,9,6


In [6]:
from re import X
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, random_state=1)

In [7]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
X_train['OverTime'] = X_train['OverTime'].replace({'Yes': 1, 'No': 0}).astype('int64')
X_test['OverTime'] = X_test['OverTime'].replace({'Yes': 1, 'No': 0}).astype('int64')
X_train

<ipython-input-7-8f7a0d90e63b>:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_train['OverTime'] = X_train['OverTime'].replace({'Yes': 1, 'No': 0}).astype('int64')
<ipython-input-7-8f7a0d90e63b>:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_test['OverTime'] = X_test['OverTime'].replace({'Yes': 1, 'No': 0}).astype('int64')


,Age,DistanceFromHome,OverTime,Education,HourlyRate,PerformanceRating,JobSatisfaction,YearsAtCompany,NumCompaniesWorked,TotalWorkingYears
464,37,1,0,3,59,3,4,6,4,12
512,27,3,0,4,54,3,4,4,0,5
480,30,12,0,4,74,3,1,1,1,1
680,36,7,0,4,65,3,4,6,2,8
1023,56,1,0,2,90,4,1,3,2,5
...,...,...,...,...,...,...,...,...,...,...
715,33,1,1,4,84,3,2,6,1,6
905,29,1,0,3,87,3,4,7,3,9
1096,40,6,1,2,38,4,4,21,1,21
235,43,16,1,3,80,4,4,17,5,22


In [8]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
scaler.fit(X_train)

# Scale the training and testing data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [9]:
from sklearn.preprocessing import OneHotEncoder

# Create a OneHotEncoder for the Department column
ohe_depart = OneHotEncoder()

# Fit the encoder to the training data
ohe_depart.fit(y_train[['Department']])

# Create two new variables by applying the encoder
# to the training and testing data
ohe_depart_train = ohe_depart.transform(y_train[['Department']]).toarray()
ohe_depart_test = ohe_depart.transform(y_test[['Department']]).toarray()
ohe_depart_train
ohe_depart_test

array([[0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       ...,
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.]])

In [10]:
# Create a OneHotEncoder for the Attrition column
ohe_attrition = OneHotEncoder()


# Fit the encoder to the training data
ohe_attrition.fit(y_train[['Attrition']])

# Create two new variables by applying the encoder
# to the training and testing data
ohe_attrition_train = ohe_attrition.transform(y_train[['Attrition']]).toarray()
ohe_attrition_test = ohe_attrition.transform(y_test[['Attrition']]).toarray()

## Part 2: Create, Compile, and Train the Model

In [11]:
# Find the number of columns in the X training data.
X_train_scaled.shape

# Create the input layer
input_layer = layers.Input(shape=(X_train_scaled.shape[1],))

# Create at least two shared layers
slayer1 = layers.Dense(units=64, activation='relu', name='shared_layer1')(input_layer)
slayer2 = layers.Dense(units=128, activation='relu', name='shared_layer2')(slayer1)

In [12]:
# Create a branch for Department
# with a hidden layer and an output layer
# Create the hidden layer
dep_branch = layers.Dense(units=32, activation='relu', name='dep_hidden')(slayer2)

# Create the output layer
dep_output = layers.Dense(units=3, activation='softmax', name='dep_output')(dep_branch)

In [13]:
# Create a branch for Attrition
# with a hidden layer and an output layer
# Create the hidden layer
att_branch = layers.Dense(units=32, activation='relu', name='att_hidden')(slayer2)

# Create the output layer
att_output = layers.Dense(units=2, activation='sigmoid',name='att_output')(att_branch)

In [14]:
# Create the model
model = Model(inputs=input_layer, outputs=[dep_output, att_output])

# Compile the model
model.compile(optimizer='adam', loss={'dep_output':'categorical_crossentropy', 'att_output':'categorical_crossentropy'},
              metrics={'dep_output':'accuracy','att_output':'accuracy'})

# Summarize the model
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 10)             │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ shared_layer1 (Dense)     │ (None, 64)             │            704 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ shared_layer2 (Dense)     │ (None, 128)            │          8,320 │ shared_layer1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dep_hidden (Dense)        │ (None, 32)             │          4,128 │ shared_layer2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ att_hidden (Dense)        │ (None, 32)             │          4,128 │ shared_layer2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dep_output (Dense)        │ (None, 3)              │             99 │ dep_hidden[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ att_output (Dense)        │ (None, 2)              │             66 │ att_hidden[0][0]       │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 17,445 (68.14 KB)

 Trainable params: 17,445 (68.14 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Train the model
model.fit(X_train_scaled, {'dep_output':ohe_depart_train, 'att_output':ohe_attrition_train}, epochs=100,
          batch_size = 32,
          validation_data=(X_test_scaled, {'dep_output':ohe_depart_test, 'att_output':ohe_attrition_test}))

In [ ]:
# Evaluate the model with the testing data
test_results = model.evaluate(X_test_scaled, {'department_output': department_test_enc, 'attrition_output': att_test_enc})
test_results

In [ ]:
# Print the accuracy for both department and attrition
print(f"Department Accuracy: {test_results[2]}")
print(f"Attrition Accuracy: {test_results[3]}")

# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. Accuracy may not be the best way to evaluate this model for predicting department because of the imbalanced values in the dataset. You could possibly use a precision, recall or F1 score to better evaluate the model.
2. Softmax for the Department class because it's a multi-class prediction. Sigmoid for the Attrition because it's a binary class.
3. Increasing the number of features or changing the features used in the X dataset may change the predictive value.